##Important library install and connection

In [2]:
import pandas as pd
import synapseclient

syn = synapseclient.Synapse()
with open('token.txt', 'r') as file:
  token = file.readline().strip()
syn.login(authToken=token)

Welcome, Humphrey Kanyoke!



##Dataset with their access codes
List of tables available in the database. Those which are of our use is marked as (o) and those which are not of our use is marked as (x). Table which might be required for discussion or for extension purpose are marked as (~)

|Table name         | Access code  | Note | Time Series | # Patients| Avg # entries per patient| Median  # entries per patient |
|-------------------|--------------|------|-------------|-----------|-------|-------|
|Day One Survey **(x)**      |syn16782072   |
|PAR-Q Survey **(x)**       | syn16782071  |
|Daily Check Survey **(o)** |syn16782070   | | Y | 17622 | 7.74 | 4.0 |
|Activity and Sleep Survey **(~)** | syn16782069| for discussion purpose | ? | 23232 | 1.07 | 1.0 |
|Risk Factor Survey **(o)** |syn16782068   | | ? | 13851 | 1.03 | 1.0 |
|Cardio Diet Survey **(x)** | syn16782067  |
|Satisfied Survey   **(x)**| syn16782066 |
|**APH Heart Age Survey (o)**|**syn16782065** | **Contains BP data** | Y | 4759 | 2.26 | 1.0 |
|Six Minute Walk Activity **(o)**| syn16782064 | | Y | 3441 | 1.98 | 1.0 |
|Demographics Survey **(o)** | syn16782063 | Contains wakeup, sleep time| Y | 7565 | 1.64 | 1.0 |
|HealthKit Data **(o)**| syn16782062 | Distance walked/run data from smart device | Y | 4920 | 23.77 | 5.0 |
|HealthKit Sleep **(o)**| syn16782061 | Sleep data from smart device| Y | 626 | 4.22 | 2.0 |
|HealthKit Workout **(o)** | syn16782060 | Workout data (mostly walking) from smart device| Y | 881 | 3.71 | 1.0 |
|Motion Tracker **(x)**| syn16782059 | Its a tmp file
|Six Minute Walk - Displacement Vectors **(x)**| syn16782058| Unknown file type


##Data Access and pre-processing


In [ ]:
query = syn.tableQuery("SELECT * FROM  	syn16782061")
raw_df = query.asDataFrame()
print(raw_df)

# Specify the columns to download
cols_to_download = ['data.csv']

# Download the CSV file
downloaded_files = syn.downloadTableColumns(query, cols_to_download)

**Clean Sleep data**

In [ ]:
import pandas as pd

def clean_sleep_record(raw_df, index):
  # Get the file handle ID of the CSV file
  file_handle_id = raw_df.iloc[index]['data.csv']

  # Get the path of the file in the cache
  file_path = syn.cache.get(file_handle_id)

  try:
    # Load the CSV file into a pandas DataFrame and group by day (summming)
    df = pd.read_csv(file_path)
    df = df[df['category.value']=='HKCategoryValueSleepAnalysisAsleep']
    df['startTime'] = pd.to_datetime(df['startTime'])
    df['value'] = df['value'] / 60    # Convert from seconds to minutes
    df['date'] = df['startTime'].dt.date
    df['bed_time'] = df['startTime'].dt.time
    df = df.drop_duplicates(subset=['startTime', 'value'])    # Remove duplicate sleep entries
    # Group by 'date', calculate sum of 'value', count the number of rows per group, and get the date of the row with the highest 'value'
    clean_df = df.groupby('date').agg({'value': ['sum', 'count', lambda x: df.loc[x.idxmax(), 'bed_time']]}).reset_index()



    clean_df.columns = ['date', 'sleep_minutes', 'awake_count', 'bed_time']

    return clean_df

  except:
    ('Bad date entry')

**Create Master sleep dataframe**

In [ ]:
from tqdm import tqdm

# Create empty df to which entries in the form of (healthCode, Date, value) will be entered
sleep = pd.DataFrame(columns=['healthCode', 'date', 'sleep_minutes', 'awake_count', 'bed_time'])

# For every clean set of sleep entries, add all the (Date, value) tuples to its corresponding 
# healthCode to create the entry to add to the main results dataframe
for i in tqdm(range(raw_df.shape[0])):
  sleep_entries = clean_sleep_record(raw_df, i)
  if sleep_entries is not None:
    sleep_entries['healthCode'] = raw_df.iloc[i]['healthCode']
    sleep = pd.concat([sleep, sleep_entries], ignore_index=True)

# Adds waking minutes and removes entries with more than 18 hours of sleep
sleep.loc['awake_minutes'] = (24 * 60) - sleep['sleep_minutes']
sleep = sleep[sleep['sleep_minutes'] < (18 * 60)]
sleep.to_csv('sleep.csv', index=False)

100%|██████████| 2644/2644 [00:23<00:00, 114.74it/s]


In [ ]:
print(sleep)

**Merge BP and Sleep Data**

In [ ]:
query = syn.tableQuery("SELECT * FROM   syn16782065")
bp = query.asDataFrame()
bp['createdOn'] = bp['createdOn'] / 1000
bp['createdOn'] = pd.to_datetime(bp['createdOn'], unit='s')
bp['date'] = bp['createdOn'].dt.date

bp = bp.merge(sleep, on=['healthCode', 'date'], how='left')

In [ ]:
print(bp[bp['sleep'].notna()])

###**Data Download and Pre-processing of HealthKit Data (Activity Data)**


**Setting up Config for download**

In [2]:
import synapseclient
syn = synapseclient.Synapse()
syn.cache.cache_root_dir = 'raw_data'    # Change cache path
with open('token.txt', 'r') as file: # fran_token.txt would work just fine
  token = file.readline().strip()
syn.login(authToken=token)


Welcome, Humphrey Kanyoke!



**Downloading all individual .csv file**\
*!! Download of 3.62 GBs of data, may take 90 minutes or more*

In [3]:
## Querying the HealthKit Data table

query0 = syn.tableQuery("select healthCode from syn16782065")
patients_list_df = query0.asDataFrame()
patients_list = patients_list_df['healthCode'].astype(str).to_list()
patients = ','.join(f"'{x}'" for x in patients_list)

query1 = "select * from syn16782062 where healthCode in ("+ patients + ")"
query1 = syn.tableQuery(query1)
raw_df_dat = query1.asDataFrame()

# # Specify the columns to download
# cols_to_download = ['data.csv']

# # Download the CSV file
# syn.downloadTableColumns(query1, cols_to_download)

**Creating Master activities dataframe**

In [73]:
import pandas as pd
from tqdm import tqdm

def extract_hk_records(raw_df_dat, index):
  '''
  Extracts the HealthKit records from the synapse database and returns a DataFrame containing
  all records for all patients
  '''
  file_handle_id = raw_df_dat.iloc[index]['data.csv']
  file_path = syn.cache.get(file_handle_id)

  df_dat = pd.read_csv(file_path)

  # Date formating
  date_format = "%Y-%m-%dT%H:%M:%S%z"  # ISO 8601 date format with timezone
  df_dat['startTime'] = pd.to_datetime(df_dat['startTime'], format=date_format, errors='coerce', utc=True )
  # Convert datetime objects to Timestamp objects and remove timezone information
  df_dat['startTime'] = df_dat['startTime'].apply(lambda x: pd.Timestamp(x).tz_localize(None) if pd.notnull(x) else pd.NaT)
  df_dat['date'] = df_dat['startTime'].dt.date

  return df_dat


def clean_hk_records(df_dat, activities_for_sum, activities_for_avg):
  '''
  Cleans the HealthKit records by transposing the 'type' column into separate columns and
  aggregating the records to a daily rate
  '''
  print('number of total entries:', df_dat.shape[0])
  # Filter out the records that are not in the list of activities to sum or average
  df_dat = df_dat[df_dat['type'].isin(activities_for_sum+activities_for_avg)].copy()
  print('filtered columns', df_dat['type'].unique())
  print('number of filtered entries:', df_dat.shape[0])
  print('unique healthCode & date entries:', df_dat.groupby(['healthCode', 'date']).ngroups)
  print('unique healthCode, date, & type entries:', df_dat.groupby(['healthCode', 'date', 'type']).ngroups)
  # Ensure that the 'value' column contains only numeric values
  df_dat['value'] = pd.to_numeric(df_dat['value'], errors='coerce').fillna(0)

  # Pivot the DataFrame so that each unique value in 'type' becomes a new column
  df_dat.reset_index(inplace=True)
  df_dat.rename(columns={'index': 'temp_index'}, inplace=True)
  df_dat = df_dat.pivot_table(index=['temp_index', 'healthCode', 'date'], 
                              columns='type', values='value').reset_index()
  df_dat = df_dat.fillna(0)
  print('pivoted shape', df_dat.shape)
  print('pivoted columns', df_dat.columns)
  print('total non-zero entries after pivoting:', df_dat.iloc[:, 3:].astype(bool).sum().sum())
  
  # Group by 'date', sum for activities_for_sum, average for activities_for_avg
  activities_for_sum = list(set(activities_for_sum) & set(df_dat.columns))
  activities_for_avg = list(set(activities_for_avg) & set(df_dat.columns))
  agg_dict = {activity: 'sum' for activity in activities_for_sum}
  agg_dict.update({activity: 'mean' for activity in activities_for_avg})
  clean_df = df_dat.groupby(['healthCode', 'date']).agg(agg_dict).reset_index()
  clean_df = clean_df.fillna(0)

  return clean_df

In [71]:
raw_hk = pd.read_csv('data/raw_hk.csv')
print(raw_hk.head())
print(raw_hk.shape)

C:\Users\frana\AppData\Local\Temp\ipykernel_10424\2638161674.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_hk = pd.read_csv('data/raw_hk.csv')


                             healthCode        date            startTime  \
0  12a38046-1512-409a-b3a1-6046e97e650e  2015-06-21  2015-06-21 18:58:49   
1  12a38046-1512-409a-b3a1-6046e97e650e  2015-06-21  2015-06-21 19:22:11   
2  12a38046-1512-409a-b3a1-6046e97e650e  2015-06-21  2015-06-21 19:28:11   
3  12a38046-1512-409a-b3a1-6046e97e650e  2015-06-21  2015-06-21 23:22:37   
4  12a38046-1512-409a-b3a1-6046e97e650e  2015-06-21  2015-06-21 23:28:37   

                     endTime                                            type  \
0  2015-06-21T13:58:51-05:00  HKQuantityTypeIdentifierDistanceWalkingRunning   
1  2015-06-21T14:28:11-05:00  HKQuantityTypeIdentifierDistanceWalkingRunning   
2  2015-06-21T14:28:14-05:00  HKQuantityTypeIdentifierDistanceWalkingRunning   
3  2015-06-21T18:28:37-05:00  HKQuantityTypeIdentifierDistanceWalkingRunning   
4  2015-06-21T18:30:27-05:00  HKQuantityTypeIdentifierDistanceWalkingRunning   

     value unit source  sourceIdentifier  
0  1.63892    m  ph

In [75]:
# Subsample hk for practicality during debugging
sub_raw_hk = raw_hk.iloc[:, :-3]
print('___________________________________________________________________')

activities_for_sum = ['HKQuantityTypeIdentifierFlightsClimbed', 'HKQuantityTypeIdentifierDistanceWalkingRunning', 
                      'HKQuantityTypeIdentifierStepCount', 'HKQuantityTypeIdentifierDistanceCycling', 
                      'HKQuantityTypeIdentifierActiveEnergyBurned']
activities_for_avg = ['HKQuantityTypeIdentifierHeartRate', 'HKQuantityTypeIdentifierBloodPressureDiastolic', 
                      'HKQuantityTypeIdentifierBloodPressureSystolic']

hk = clean_hk_records(sub_raw_hk, activities_for_sum, activities_for_avg)
# print number of rows with all 0 values for all columns except for date and healthCode
print('total non-zero entries after aggregating:', hk.iloc[:, 3:].astype(bool).sum())
print('number of rows in hk', hk.shape[0])
print('rows with at least 1 non-zero entry', hk.iloc[:, 3:].ne(0).any(axis=1).sum())
print('total non-zero entries after aggregating:', hk.iloc[:, 3:].astype(bool).sum().sum())

___________________________________________________________________
number of total entries: 18768522
filtered columns ['HKQuantityTypeIdentifierDistanceWalkingRunning'
 'HKQuantityTypeIdentifierStepCount'
 'HKQuantityTypeIdentifierFlightsClimbed'
 'HKQuantityTypeIdentifierHeartRate'
 'HKQuantityTypeIdentifierDistanceCycling'
 'HKQuantityTypeIdentifierBloodPressureSystolic'
 'HKQuantityTypeIdentifierBloodPressureDiastolic'
 'HKQuantityTypeIdentifierActiveEnergyBurned']
number of filtered entries: 18613281
unique healthCode & date entries: 51123
unique healthCode, date, & type entries: 127223
pivoted shape (18613223, 11)
pivoted columns Index(['temp_index', 'healthCode', 'date',
       'HKQuantityTypeIdentifierActiveEnergyBurned',
       'HKQuantityTypeIdentifierBloodPressureDiastolic',
       'HKQuantityTypeIdentifierBloodPressureSystolic',
       'HKQuantityTypeIdentifierDistanceCycling',
       'HKQuantityTypeIdentifierDistanceWalkingRunning',
       'HKQuantityTypeIdentifierFlightsC

In [76]:
hk.to_csv('data/healthkit.csv', index=False)

In [ ]:
hk = pd.DataFrame(columns=['healthCode', 'date'])
for i in tqdm(range(raw_df_dat.shape[0])):
  hk_entries = extract_hk_records(raw_df_dat, i)
  if hk_entries is not None:
    hk_entries['healthCode'] = raw_df_dat.iloc[i]['healthCode']
    hk = pd.concat([hk, hk_entries], ignore_index=True)

**Merging BP and activities data**

In [ ]:
query = syn.tableQuery("SELECT * FROM   syn16782065")
bp = query.asDataFrame()
bp['createdOn'] = bp['createdOn'] / 1000
bp['createdOn'] = pd.to_datetime(bp['createdOn'], unit='s')
bp['date'] = bp['createdOn'].dt.date

bp_activities = bp.merge(df_activities, on=['healthCode', 'date'], how='left')

In [ ]:
bp_activities

###**Data Download and Pre-processing of HealthKit Workout**


**Setting up config for download**

In [ ]:
syn.cache.cache_root_dir = '/content/workout_data'    # Change cache path

**Downloading all individual .csv file**
*Took 3m 21s to download the data*

In [ ]:
query_wo = syn.tableQuery("select healthCode from syn16782065")
patients_list_df = query_wo.asDataFrame()
patients_list = patients_list_df['healthCode'].astype(str).to_list()
patients = ','.join(f"'{x}'" for x in patients_list)

query_wo = "select * from syn16782060 where healthCode in ("+ patients + ")"
query_wo = syn.tableQuery(query_wo)
raw_df_wo = query_wo.asDataFrame()

# Specify the columns to download
cols_to_download = ['data.csv']

# Download the CSV file
syn.downloadTableColumns(query_wo, cols_to_download)

**Creating workout dataframe**

In [ ]:
bad_records_wo = []
def clean_data_records(raw_df_wo, index):
    file_handle_id = raw_df_wo.iloc[index]['data.csv']
    file_path = syn.cache.get(file_handle_id)

    try:
      df_dat = pd.read_csv(file_path)
      df_dat['startTime'] = pd.to_datetime(df_dat['startTime'])
      df_dat['endTime'] = pd.to_datetime(df_dat['endTime'])

      df_dat = df_dat[df_dat['workoutType'].str.contains('HKWorkoutActivityType')] # deleting rows which contains junt activity type
      df_dat = df_dat.drop(['type', 'total.distance', 'unit', 'source', 'sourceIdentifier', 'metadata'], axis=1) # Dropping unnecessary columns
      df_dat = df_dat.drop_duplicates(subset=['startTime', 'endTime']) # removing redundant rows
      #df_dat_sum = df_dat.groupby('date')['energy.consumed'].sum().reset_index()
      df_dat = df_dat[pd.to_numeric(df_dat['energy.consumed'], errors='coerce').notnull()] # deleting values having non-numeric energy.consumed value
      return df_dat

    except:
        bad_records_wo.append(raw_df_wo.iloc[index]['recordId'])

df_workout = pd.DataFrame()

# For every clean set of sleep entries, add all the (Date, value) tuples to its corresponding healthCode to create the entry to add to the main results dataframe
for i in range(raw_df_wo.shape[0]):
#for i in range(209): #To run a subset
    data_entries = clean_data_records(raw_df_wo, i)
    #print(data_entries)
    if data_entries is not None:
        data_entries['healthCode'] = raw_df_wo.iloc[i]['healthCode']
        df_workout = pd.concat([df_workout, data_entries], ignore_index=True)
        print(i, " of ", raw_df_wo.shape[0], " processed")

df_workout.to_csv('workout.csv', index=False)


Add rolling K averaging to populate missing values in dataframe